In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

In [ ]:
# poetry add bs4

In [3]:
import os
from langchain_community.document_loaders import WebBaseLoader

# 웹 요청을 위한 USER_AGENT 환경 변수 설정 (필요한 경우)
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"

# 환경 변수 확인
print(f"현재 설정된 USER_AGENT: {os.environ.get('USER_AGENT')}")

# 웹페이지 URL 지정  https://ko.wikipedia.org/wiki/축구_경기_규칙
url = 'https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC_%EA%B2%BD%EA%B8%B0_%EA%B7%9C%EC%B9%99'

# WebBaseLoader 초기화 및 데이터 로드
loader = WebBaseLoader(url)
docs = loader.load()

# 로드된 문서 확인
print(type(docs), len(docs))
#print(docs)
print(type(docs[0]))  # <class 'langchain_core.documents.Document'>
print(len(docs[0].page_content))
print(docs[0].page_content[5000:5500])
print(docs[0].metadata)

현재 설정된 USER_AGENT: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36
<class 'list'> 1
<class 'langchain_core.documents.base.Document'>
34666
이(골 라인) : 최소 45m(50야드), 최대 90m(100야드)
4. 국제 경기에서의 치수(Dimensions for International Matches)[원본 편집]
길이(터치 라인) : 최소 100m(110야드), 최대 110m(120야드)
길이(골 라인) : 최소 64m(70야드), 최대 75m(80야드)
대회는 상기 차원 내에서 골 라인과 터치 라인의 길이를 결정할 수 있다.
측정은 선이 둘러싸는 영역의 일부이므로 선 외부에서 이루어진다.
페널티 마크는 마크 중앙에서 골 라인 뒤쪽 가장자리까지 측정된다
5. 골 에어리어(The Goal Area)[원본 편집]
각 골대 안쪽에서 5.5m(6야드) 떨어진 골 라인에 직각으로 두 개의 선이 그려진다. 이 선은 경기장까지 5.5m(6야드) 연장되며 골라인과 평행하게 그려진 선으로 연결된다. 이 라인과 골 라인으로 둘러싸인 영역이 골 에어리어이다.
6. 페널티 에어리어(The Penalty Area)[원본 편집]
각 골대 안쪽에
{'source': 'https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC_%EA%B2%BD%EA%B8%B0_%EA%B7%9C%EC%B9%99', 'title': '축구 경기 규칙 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

# 문서 분할
splits = text_splitter.split_documents(docs)

# 분할된 문서 확인
print(type(splits), len(splits))  # 총 몇 개의 청크가 생성되었는지 확인
print(type(splits[0])) # Document
print(splits[0].page_content[:20])  # 첫 번째 청크의 일부 출력
# 열번째 청크의 내용 출력
print(splits[10].page_content[:20])
# 열번째 청크의 메타데이터 출력
print(splits[10].metadata)

<class 'list'> 16
<class 'langchain_core.documents.base.Document'>
축구 경기 규칙 - 위키백과, 우리 
경기는 주심과 참가한 두 팀이 상호 
{'source': 'https://ko.wikipedia.org/wiki/%EC%B6%95%EA%B5%AC_%EA%B2%BD%EA%B8%B0_%EA%B7%9C%EC%B9%99', 'title': '축구 경기 규칙 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}


In [5]:
from langchain_community.vectorstores import FAISS  # 벡터 저장소 라이브러리
from langchain_openai import OpenAIEmbeddings # OpenAI의 임베딩(Embedding) 모델

# 1. FAISS 벡터 저장소 생성
# - documents: 텍스트 데이터를 벡터화 하여 저장할 문서 리스트
# - embedding: 문서를 벡터로 변환하는 OpenAI Embeddings 모델 사용
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 2. 유사 문서 검색 (Similarity Search)
# - "경기장 표시에 대해서 설명해주세요."라는 쿼리에 대해,
# - FAISS 벡터 저장소에서 가장 유사한 문서를 검색함.
docs = vectorstore.similarity_search("경기장 표시에 대해서 설명해주세요.")

# 3. 검색된 문서의 타입과 개수 출력
print(type(docs), len(docs))
# 4. 검색된 첫 번째 문서 내용 출력
print(docs[0].page_content)

<class 'list'> 4
2. 경기장 표시(Field Markings)[원본 편집]
경기장은 직사각형이어야 하며 위험하지 않아야 하는 연속적인 선으로 표시되어야 한다. 위험하지 않은 경우 인공 경기장 재료를 자연 필드의 필드 표시에 사용할 수 있다. 이 선은 경계가 되는 영역에 속한다.
경기장에는 규칙1에 명시된 라인만 표시되어야 한다. 인공 표면이 사용되는 경우, 다른 색이 있고 축구 라인과 명확하게 구별되는 다른 라인이 허용된다.
두 개의 긴 경계선은 터치 라인이다. 두 개의 짧은 라인은 골 라인이다.
경기장은 두 개의 터치 라인의 중간 지점을 연결하는 중간 선으로 두 부분으로 나뉜다.
중앙 표시는 중간이라는 뜻이며. 반경이 9.15m(10야드)인 원이 그 주위에 표시된다.
코너 호에서 골 라인과 터치 라인에 직각으로 9.15m(10야드) 경기장 밖에서 마크를 할 수 있다.
모든 선은 너비가 같아야하며 12cm(5인치)를 넘지 않아야 한다. 골라인은 골 포스트 및 크로스바와 너비가 같아야 한다.
경기장에서 승인되지 않은 마크일 경우, 선수는 비신사적 행동에 대해 주의를 기울여야 한다. 심판이 경기 중에 이것이 행해지는 것을 알아차리면, 선수는 경기 중, 볼이 플레이 아웃 될 때 주의를 받는다.
3. 치수(Dimensions)[원본 편집]
터치 라인은 골 라인보다 길어야 한다.
길이(터치 라인) : 최소 90m(100야드), 최대 120m(130야드)
길이(골 라인) : 최소 45m(50야드), 최대 90m(100야드)
4. 국제 경기에서의 치수(Dimensions for International Matches)[원본 편집]
길이(터치 라인) : 최소 100m(110야드), 최대 110m(120야드)
길이(골 라인) : 최소 64m(70야드), 최대 75m(80야드)
대회는 상기 차원 내에서 골 라인과 터치 라인의 길이를 결정할 수 있다.
측정은 선이 둘러싸는 영역의 일부이므로 선 외부에서 이루어진다.
페널티 마크는 마크 중앙에서 골 라인 뒤쪽 가장자리

In [6]:
from langchain_openai import ChatOpenAI  # OpenAI LLM(대화형 언어 모델)
from langchain_core.prompts import ChatPromptTemplate  # 프롬프트 템플릿
from langchain_core.runnables import RunnablePassthrough  # 입력을 그대로 전달하는 유틸리티
from langchain_core.output_parsers import StrOutputParser  # LLM 응답을 문자열로 변환하는 파서
from pprint import pprint

# 검색 개수 제한 설정
# - 벡터 저장소(vectorstore)에서 관련성이 높은 문서 최대 6개를 검색하도록 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})  

def format_docs(docs):
    summaries = [f"출처: {doc.metadata.get('source', '알 수 없음')}\n" + doc.page_content[:300] + "..." for doc in docs]
    return "\n\n".join(summaries)    

# 프롬프트 템플릿 설정
# - 모델이 주어진 `context`(검색된 문서)만을 참고하여 질문에 답변하도록 유도하는 프롬프트 템플릿
# - {context}: 검색된 문서 요약이 삽입될 자리
# - {question}: 사용자의 질문이 삽입될 자리
template = '''당신은 제공된 컨텍스트를 기반으로 질문에 답하는 AI 어시스턴트입니다. 
반드시 컨텍스트 내 정보를 활용하여 정확하고 신뢰할 수 있는 답변을 제공하세요.

[컨텍스트]
{context}

[질문]
{question}

[답변]
'''

# - 위에서 정의한 템플릿을 사용하여 LangChain의 프롬프트 객체 생성
prompt = ChatPromptTemplate.from_template(template)  

# LLM 모델 설정
# - OpenAI의 `gpt-3.5-turbo-0125` 모델 사용 (정확도를 높이기 위해 temperature=0 설정)
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# RAG 체인 설정
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  
    # - `retriever`를 통해 검색된 문서를 `format_docs()` 함수로 가공하여 `context`로 전달
    # - `question`은 변형 없이 그대로 전달
    | prompt  # - 위에서 정의한 `ChatPromptTemplate`을 적용
    | model   # - OpenAI GPT-3.5 모델을 사용해 응답 생성
    | StrOutputParser()  # - 모델의 응답을 문자열로 변환
)

# 실행 (사용자 질문을 입력으로 받아 RAG 체인 실행)
# - "경기장 표시에 대해서 설명해주세요."라는 질문을 LLM에 전달하여 답변을 생성
response = rag_chain.invoke("경기장 표시에 대해서 설명해주세요.")  

# 최종 응답 출력
print(f" 모델 응답:\n")
pprint(response)

 모델 응답:

('경기장은 직사각형이어야 하며 위험하지 않아야 하는 연속적인 선으로 표시되어야 합니다. 경기장에는 규칙에 명시된 라인만 표시되어야 하며, '
 '인공 표면이 사용되는 경우 다른 색이 있고 축구 라인과 명확하게 구별되는 다른 라인이 허용됩니다. 두 개의 긴 경계선은 터치 라인이고, '
 '두 개의 짧은 라인은 골 라인입니다. 경기장은 두 개의 터치 라인의 중간 지점을 연결해야 합니다.')
